# Taller 3

**Autor:** Juan Pablo Gaviria

## Deteccion de Topicos con genSim

Se realizará la detección de Topicos con genSim y LDA

In [2]:
import pandas as pd
import numpy as np
from gensim.corpora import Dictionary
import snowballstemmer
from nltk.corpus import stopwords
from string import ascii_lowercase
import itertools
from collections import Counter
import re
import multiprocessing as mp
import time
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis
import gensim
import nltk
from pyLDAvis import gensim

### Lectura de la informacion

In [3]:
originalData = pd.read_csv('/home/jgaviria/Workspace/DataScience/datasets/metadata.csv')
originalData

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128487,a11jyui1,NaN,Elsevier; PMC,Impact of BSE on the biotechnology industry — ...,10.1016/s0958-2118(04)00238-1,PMC7148827,NaN,els-covid,This feature briefly looks at filtration techn...,2004-10-31,"Atkinson, Simon",Membrane Technology,NaN,NaN,NaN,NaN,NaN,https://www.sciencedirect.com/science/article/...,109580667.0
128488,9bma9y0q,44959e4505dcc59575f4989de8abf3fcc7e48415,Medline; PMC,Evolving Gene Targets and Technology in Influe...,10.1007/s40291-013-0040-9,PMC7100497,23686537.0,no-cc,Influenza viruses cause recurring epidemic out...,2013-05-18,"Malanoski, Anthony P.; Lin, Baochuan",Mol Diagn Ther,NaN,NaN,NaN,document_parses/pdf_json/44959e4505dcc59575f49...,document_parses/pmc_json/PMC7100497.xml.json,NaN,17361009.0
128489,50xhia4r,8922380f61b6fe160ab0550a674816f1852a843d,Medline; PMC; WHO,How to train health personnel to protect thems...,10.3352/jeehp.2020.17.10,PMC7162995,32150796.0,cc-by,NaN,2020-03-07,"Huh, Sun",J Educ Eval Health Prof,NaN,#6263,NaN,document_parses/pdf_json/8922380f61b6fe160ab05...,document_parses/pmc_json/PMC7162995.xml.json,NaN,212650605.0
128490,97h1fz34,ea20a6b97f98ce4bdddfa6408da5ddf3dd06835e,Medline; PMC,Epidemic spreading in complex networks,10.1007/s11467-008-0027-x,PMC7111544,32288753.0,no-cc,The study of epidemic spreading in complex net...,2008-07-08,"Zhou, Jie; Liu, Zong-hua",Front Phys China,NaN,NaN,NaN,document_parses/pdf_json/ea20a6b97f98ce4bdddfa...,NaN,NaN,121665618.0


### Preparacion de la informacion

In [4]:
data = originalData.drop(['sha','doi','pmcid','pubmed_id','license','mag_id','who_covidence_id','arxiv_id','pdf_json_files','pmc_json_files','url','s2_id'], axis=1)
data

,cord_uid,source_x,title,abstract,publish_time,authors,journal
0,ug7v899j,PMC,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis
1,02tnwd4m,PMC,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res
2,ejv2xln0,PMC,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res
3,2b73a28n,PMC,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res
4,9785vg6d,PMC,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res
...,...,...,...,...,...,...,...
128487,a11jyui1,Elsevier; PMC,Impact of BSE on the biotechnology industry — ...,This feature briefly looks at filtration techn...,2004-10-31,"Atkinson, Simon",Membrane Technology
128488,9bma9y0q,Medline; PMC,Evolving Gene Targets and Technology in Influe...,Influenza viruses cause recurring epidemic out...,2013-05-18,"Malanoski, Anthony P.; Lin, Baochuan",Mol Diagn Ther
128489,50xhia4r,Medline; PMC; WHO,How to train health personnel to protect thems...,NaN,2020-03-07,"Huh, Sun",J Educ Eval Health Prof
128490,97h1fz34,Medline; PMC,Epidemic spreading in complex networks,The study of epidemic spreading in complex net...,2008-07-08,"Zhou, Jie; Liu, Zong-hua",Front Phys China


In [5]:
# Se crea nuevo dataframe unicamente con el id y el texto a evaluar
texto = pd.DataFrame({'id':data.cord_uid, 'texto':data.title+" "+data.abstract})
texto

,id,texto
0,ug7v899j,Clinical features of culture-proven Mycoplasma...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...
2,ejv2xln0,Surfactant protein-D and pulmonary host defens...
3,2b73a28n,Role of endothelin-1 in lung disease Endotheli...
4,9785vg6d,Gene expression in epithelial cells in respons...
...,...,...
128487,a11jyui1,Impact of BSE on the biotechnology industry — ...
128488,9bma9y0q,Evolving Gene Targets and Technology in Influe...
128489,50xhia4r,NaN
128490,97h1fz34,Epidemic spreading in complex networks The stu...


In [6]:
# Se eliminan filas que no tengan ni titulo ni abstract
texto = texto.dropna()
texto

,id,texto
0,ug7v899j,Clinical features of culture-proven Mycoplasma...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...
2,ejv2xln0,Surfactant protein-D and pulmonary host defens...
3,2b73a28n,Role of endothelin-1 in lung disease Endotheli...
4,9785vg6d,Gene expression in epithelial cells in respons...
...,...,...
128486,jvl63652,Dual Analysis of the Murine Cytomegalovirus an...
128487,a11jyui1,Impact of BSE on the biotechnology industry — ...
128488,9bma9y0q,Evolving Gene Targets and Technology in Influe...
128490,97h1fz34,Epidemic spreading in complex networks The stu...


### Preparacion del texto

In [7]:
# Preparacion stopwords
# Se crea un listado de stopwords con nltk y se expande a su raiz con stem
stemmer = snowballstemmer.EnglishStemmer()
stop = stopwords.words('english')
stop.extend(['may','also','zero','one','two','three','four','five','six','seven','eight','nine','ten','across','among','beside','however','yet','within']+list(ascii_lowercase))
stopList = stemmer.stemWords(stop)
stopList = set(stopList)
stop = set(sorted(stop + list(stopList)))

In [8]:
# Limpieza de caracteres especiales
texto['texto'].replace('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ',inplace=True,regex=True)

<>:2: DeprecationWarning: invalid escape sequence \[
<>:2: DeprecationWarning: invalid escape sequence \[
<>:2: DeprecationWarning: invalid escape sequence \[
<ipython-input-8-61b0bb2beb8c>:2: DeprecationWarning: invalid escape sequence \[
  texto['texto'].replace('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ',inplace=True,regex=True)


In [9]:
# Tokenizaicion, Stemming y remocion de stopwords
wordlist = filter(None, " ".join(list(set(list(itertools.chain(*texto['texto'].str.split(' ')))))).split(" "))
texto['stemmed_text'] = [' '.join(filter(None,filter(lambda word: word not in stop, line))) for line in texto['texto'].str.lower().str.split(' ')]
texto



,id,texto,stemmed_text
0,ug7v899j,Clinical features of culture proven Mycoplasma...,clinical features culture proven mycoplasma pn...
1,02tnwd4m,Nitric oxide a pro inflammatory mediator in l...,nitric oxide pro inflammatory mediator lung di...
2,ejv2xln0,Surfactant protein D and pulmonary host defens...,surfactant protein pulmonary host defense surf...
3,2b73a28n,Role of endothelin in lung disease Endotheli...,role endothelin lung disease endothelin et ami...
4,9785vg6d,Gene expression in epithelial cells in respons...,gene expression epithelial cells response pneu...
...,...,...,...
128486,jvl63652,Dual Analysis of the Murine Cytomegalovirus an...,dual analysis murine cytomegalovirus host cell...
128487,a11jyui1,Impact of BSE on the biotechnology industry — ...,impact bse biotechnology industry — detection ...
128488,9bma9y0q,Evolving Gene Targets and Technology in Influe...,evolving gene targets technology influenza det...
128490,97h1fz34,Epidemic spreading in complex networks The stu...,epidemic spreading complex networks study epid...


In [10]:
# Se eliminan las palabras que no aparezcan por lo menos 5 veces en todos los documentos
cuenta_minima = 5
str_frequencies = pd.DataFrame(list(Counter(filter(None,list(itertools.chain(*texto['stemmed_text'].str.split(' '))))).items()),columns=['word','count'])
low_frequency_words = set(str_frequencies[str_frequencies['count'] < cuenta_minima]['word'])
texto['stemmed_text'] = [' '.join(filter(None,filter(lambda word: word not in low_frequency_words, line))) for line in texto['stemmed_text'].str.split(' ')]
texto

,id,texto,stemmed_text
0,ug7v899j,Clinical features of culture proven Mycoplasma...,clinical features culture proven mycoplasma pn...
1,02tnwd4m,Nitric oxide a pro inflammatory mediator in l...,nitric oxide pro inflammatory mediator lung di...
2,ejv2xln0,Surfactant protein D and pulmonary host defens...,surfactant protein pulmonary host defense surf...
3,2b73a28n,Role of endothelin in lung disease Endotheli...,role endothelin lung disease endothelin et ami...
4,9785vg6d,Gene expression in epithelial cells in respons...,gene expression epithelial cells response pneu...
...,...,...,...
128486,jvl63652,Dual Analysis of the Murine Cytomegalovirus an...,dual analysis murine cytomegalovirus host cell...
128487,a11jyui1,Impact of BSE on the biotechnology industry — ...,impact bse biotechnology industry — detection ...
128488,9bma9y0q,Evolving Gene Targets and Technology in Influe...,evolving gene targets technology influenza det...
128490,97h1fz34,Epidemic spreading in complex networks The stu...,epidemic spreading complex networks study epid...


In [11]:
# Funcion para Stemming paralelo
def parStemming(texto):
    return " ".join(stemmer.stemWords(re.sub('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ', texto).split(' ')))

<>:3: DeprecationWarning: invalid escape sequence \[
<>:3: DeprecationWarning: invalid escape sequence \[
<>:3: DeprecationWarning: invalid escape sequence \[
<ipython-input-11-e5f661c12f12>:3: DeprecationWarning: invalid escape sequence \[
  return " ".join(stemmer.stemWords(re.sub('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ', texto).split(' ')))


In [12]:
# Se realiza Stemming
t0 = time.time()
pool = mp.Pool()
texto['stemmed_text'] = pool.map(parStemming,texto['stemmed_text'])
pool.close()
print('Tiempo tomado: ' + str(time.time()-t0))

Tiempo tomado: 252.25737190246582


### Representacion de los documentos

In [13]:
# Funcion para tokenizacion paralela
def parToken(texto):
    return nltk.word_tokenize(texto)

In [14]:
# Tokenizacion
t0 = time.time()
pool = mp.Pool()
#texto['tokens'] = texto.apply(lambda row: nltk.word_tokenize(row['stemmed_text']), axis=1)
texto['tokens'] = pool.map(parToken, texto['stemmed_text'])
pool.close()
print('Tiempo tomado: ' + str(time.time()-t0))
texto

Tiempo tomado: 13.787389278411865


,id,texto,stemmed_text,tokens
0,ug7v899j,Clinical features of culture proven Mycoplasma...,clinic featur cultur proven mycoplasma pneumon...,"[clinic, featur, cultur, proven, mycoplasma, p..."
1,02tnwd4m,Nitric oxide a pro inflammatory mediator in l...,nitric oxid pro inflammatori mediat lung disea...,"[nitric, oxid, pro, inflammatori, mediat, lung..."
2,ejv2xln0,Surfactant protein D and pulmonary host defens...,surfact protein pulmonari host defens surfact ...,"[surfact, protein, pulmonari, host, defens, su..."
3,2b73a28n,Role of endothelin in lung disease Endotheli...,role endothelin lung diseas endothelin et amin...,"[role, endothelin, lung, diseas, endothelin, e..."
4,9785vg6d,Gene expression in epithelial cells in respons...,gene express epitheli cell respons pneumovirus...,"[gene, express, epitheli, cell, respons, pneum..."
...,...,...,...,...
128486,jvl63652,Dual Analysis of the Murine Cytomegalovirus an...,dual analysi murin cytomegalovirus host cell t...,"[dual, analysi, murin, cytomegalovirus, host, ..."
128487,a11jyui1,Impact of BSE on the biotechnology industry — ...,impact bse biotechnolog industri — detect risk...,"[impact, bse, biotechnolog, industri, —, detec..."
128488,9bma9y0q,Evolving Gene Targets and Technology in Influe...,evolv gene target technolog influenza detect i...,"[evolv, gene, target, technolog, influenza, de..."
128490,97h1fz34,Epidemic spreading in complex networks The stu...,epidem spread complex network studi epidem spr...,"[epidem, spread, complex, network, studi, epid..."


In [15]:
# BoW
diccionario = Dictionary(texto.tokens)
print('Longitud del BoW: ' + str(len(diccionario)))

Longitud del BoW: 41245


In [16]:
# Matriz de documentos vs terminos
t0 = time.time()
pool = mp.Pool(mp.cpu_count()-1)
doc_term_matrix = pool.map(diccionario.doc2bow, [sentence for sentence in texto.tokens])
pool.close()
print('Tiempo tomado: '+str(time.time()-t0))

Tiempo tomado: 6.516019821166992


### Aplicacion del modelo (LDA)

In [ ]:
t0 = time.time()
lda_model = LdaMulticore(doc_term_matrix, num_topics=5, id2word=diccionario, workers=mp.cpu_count())
print('Tiempo tomado: '+str(time.time()-t0))

### Visualizacion del modelo

In [ ]:
t0 = time.time()
pyLDAvis.enable_notebook()
vis = gensim.prepare(lda_model, doc_term_matrix, diccionario, sort_topics=False)
print('Tiempo tomado: '+str(time.time()-t0))
vis

In [ ]:
# Guardar el modelo
pyLDAvis.save_html(vis, 'gensimLDAOpt.html')

### Aplicación del modelo

Se aplica el modelo para clasificar todos los documentos. Esta clasificacion se utiliza para el clasificador supervisado

In [ ]:
# Funcion para clasificacion paralela
def clasificarDoc(documento):
    textBow = diccionario.doc2bow(documento)
    distribucion = lda_model.get_document_topics(textBow)
    topico = max(distribucion, key=lambda x:x[1])[0]
    return topico


In [ ]:
# Clasificar todos los documentos
t0 = time.time()
pool = mp.Pool()
texto['Topico'] = pool.map(clasificarDoc, texto.tokens)
pool.close()
print('Tiempo tomado: '+str(time.time()-t0))
texto

In [ ]:
# Se crea Data Frame de salida con el id del documento y el topico
outClasificacion = pd.DataFrame({'docId':texto.id,'topico':texto.Topico})
outClasificacion.to_pickle('docTopicDetection.pkl')